In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Data_for_UCI_named.csv')
data.head(3)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable


In [3]:
data.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [4]:
x = data.drop(['stab', 'stabf'], axis =1)
y = data['stabf']

In [5]:
#Split data into training and testing set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=1)

In [6]:
#Use Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_normalized = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test_normalized = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

In [7]:
#Train model with a Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(random_state=1)
random_forest_model.fit(x_train_normalized, y_train)

random_forest_pred = random_forest_model.predict(x_test_normalized)

# Question 4:

In [8]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, random_forest_pred) 
round(accuracy, 4)

0.928

# Question 9 & 10

In [9]:
#Train model with ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier
extra = ExtraTreesClassifier(random_state =1)
extra.fit(x_train_normalized, y_train)

extra_pred = extra.predict(x_test_normalized)

accuracy = accuracy_score(y_test, extra_pred) 
round(accuracy, 4)

0.926

In [12]:
#Question 10
n_estimators = [100,300,500,1000]
max_features = ['auto', 'log2', None]
min_samples_split = [2,5,7]
min_samples_leaf = [4,6,8]

param_grid = {'n_estimators':n_estimators,
             'max_features': max_features,
             'min_samples_split': min_samples_split,
             'min_samples_leaf': min_samples_leaf}

print(param_grid)

extra_mod = ExtraTreesClassifier(random_state = 1)
from sklearn.model_selection import RandomizedSearchCV
Random_df= RandomizedSearchCV(estimator = extra_mod,
                              param_distributions = param_grid,
                              cv=5, 
                              n_iter=10,
                              scoring = 'accuracy',
                              n_jobs = -1,
                              verbose = 1,
                              random_state = 1)

Random_df.fit(x_train_normalized, y_train)
print(Random_df.best_params_)

{'n_estimators': [100, 300, 500, 1000], 'max_features': ['auto', 'log2', None], 'min_samples_split': [2, 5, 7], 'min_samples_leaf': [4, 6, 8]}
Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features': None}


In [13]:
extra_mod = ExtraTreesClassifier(n_estimators= 500,min_samples_split= 5,min_samples_leaf= 6,max_features= None)
extra_mod.fit(x_train_normalized, y_train)

extra_mod_pred = extra_mod.predict(x_test_normalized)

accuracy = accuracy_score(y_test, extra_pred) 
round(accuracy, 4)

0.926

In [14]:
#No change between the accuracy of the new optimal model and the intial extratreesclassifier model

# Question 11:

In [15]:
import lightgbm as ltb
model = ltb.LGBMClassifier(random_state=1)
model.fit(x_train_normalized, y_train)

mod_pred = model.predict(x_test_normalized)

accuracy = accuracy_score(y_test, mod_pred) 
round(accuracy, 4)

0.9365

# Question 14:

In [17]:
extra_mod.feature_importances_

array([0.13556943, 0.13834602, 0.13248348, 0.13412825, 0.00523114,
       0.00722736, 0.00712177, 0.00675999, 0.10357194, 0.10737321,
       0.11225432, 0.10993308])

In [19]:
x_train_normalized.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4'],
      dtype='object')

In [21]:
def get_feature_importance(extra_mod, feat, col_name):
    feature_importance = pd.Series(extra_mod.feature_importances_, x_train_normalized.columns).sort_values()
    feature_importance_df = pd.DataFrame(feature_importance).reset_index()
    feature_importance_df.columns = ['x_train_normalized', col_name]
    feature_importance_df[col_name].round(2)
    return feature_importance_df
extraTrees_feature_importance = get_feature_importance(extra_mod, x_train_normalized, 'Feature_importance')
feature_impo = extraTrees_feature_importance
feature_impo.sort_values('Feature_importance')

,x_train_normalized,Feature_importance
0,p1,0.005231
1,p4,0.006760
2,p3,0.007122
3,p2,0.007227
4,g1,0.103572
5,g2,0.107373
6,g4,0.109933
7,g3,0.112254
8,tau3,0.132483
9,tau4,0.134128


# Question 16:

In [23]:
from xgboost import XGBClassifier
XGBoost_model = XGBClassifier(random_state=1)
XGBoost_model.fit(x_train_normalized, y_train)

xgboost_model_pred = XGBoost_model.predict(x_test_normalized)

accuracy = accuracy_score(y_test, xgboost_model_pred) 
round(accuracy, 4)

C:\Users\OMNL\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:09:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.946